In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ['LANGCHAIN_PROJECT'] = "BuffetBot"
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
import ssl
from urllib.request import urlopen
import certifi
import json
from langchain_core.tools import tool
from typing import List, Optional, TypedDict, Annotated

class FinancialRatios(TypedDict):
    symbol: str
    date: str
    operatingCashFlowPerShare: Optional[float]
    interestCoverageRatio: Optional[float]
    workingCapital: Optional[float]
    daysSalesOutstanding: Optional[float]
    bookValuePerShare: Optional[float]
    dividendYield: Optional[float]
    currentRatio: Optional[float]
    payablesTurnover: Optional[float]

class FinancialKeyMetrics(TypedDict):
    symbol: str
    date: str
    grossProfitMargin: Optional[float]
    debtEquityRatio: Optional[float]
    debtRatio: Optional[float]
    operatingProfitMargin: Optional[float]
    netProfitMargin: Optional[float]
    returnOnAssets: Optional[float]
    returnOnEquity: Optional[float]
    returnOnCapitalEmployed: Optional[float]
    totalDebtToCapitalization: Optional[float]
    cashFlowCoverageRatios: Optional[float]
    quickRatio: Optional[float]
    cashRatio: Optional[float]
    assetTurnover: Optional[float]
    inventoryTurnover: Optional[float]
    receivablesTurnover: Optional[float]
    cashConversionCycle: Optional[float]
    priceEarningsRatio: Optional[float]
    daysOfPayablesOutstanding: Optional[float]

# Function 1
@tool
def get_financial_key_metrics(ticker: str, observations: int = 1) -> List[FinancialRatios]:
    """
    Retrieve and filter key financial metrics for a specified company.

    This function fetches financial data from the Financial Modeling Prep API
    and filters it to include specific key metrics. It's designed to provide
    a focused set of financial indicators for analysis.

    Parameters:
    ticker (str): The stock symbol of the company (e.g., "AAPL" for Apple Inc.).
    observations (int, optional): The number of most recent periods to return. 
                                  Defaults to 1.

    Returns:
    List[FinancialRatios]: A list of FinancialRatio objects, each containing key metrics
                          for the specified number of observations.
    """
    api_key = os.getenv('FINANCIAL_MODELING_PREP_API_KEY')
    url = f"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?apikey={api_key}"
    
    context = ssl.create_default_context(cafile=certifi.where())
    
    with urlopen(url, context=context) as response:
        data = response.read().decode("utf-8")
    
    json_data = json.loads(data)
    
    filtered_data = []
    for item in json_data:
        filtered_item = FinancialRatios(
            symbol=item['symbol'],
            date=item['date'],
            operatingCashFlowPerShare=item.get('operatingCashFlowPerShare'),
            interestCoverageRatio=item.get('interestCoverage'),
            workingCapital=item.get('workingCapital'),
            daysSalesOutstanding=item.get('daysSalesOutstanding'),
            bookValuePerShare=item.get('bookValuePerShare'),
            dividendYield=item.get('dividendYield'),
            currentRatio=item.get('currentRatio'),
            payablesTurnover=item.get('payablesTurnover')
        )
        filtered_data.append(filtered_item)
    
    return filtered_data[:observations]

# Function 2
@tool
def get_financial_ratios(ticker: str, observations: int = 1) -> List[FinancialKeyMetrics]:
    """
    Retrieve and filter financial ratios for a specified company.

    This function fetches financial ratio data from the Financial Modeling Prep API
    and filters it to include specific key ratios. It's designed to provide
    a focused set of financial indicators for analysis.

    Parameters:
    ticker (str): The stock symbol of the company (e.g., "AAPL" for Apple Inc.).
    api_key (str): Your Financial Modeling Prep API key for authentication.
    observations (int, optional): The number of most recent periods to return. 
                                  Defaults to 1.

    Returns:
    List[FinancialKeyMetrics]: A list of FinancialKeyMetric objects, each containing key ratios
                              for the specified number of observations.
    """
    api_key = os.getenv('FINANCIAL_MODELING_PREP_API_KEY')
    url = f"https://financialmodelingprep.com/api/v3/ratios/{ticker}?&apikey={api_key}"
    
    context = ssl.create_default_context(cafile=certifi.where())
    
    with urlopen(url, context=context) as response:
        data = response.read().decode("utf-8")
    
    json_data = json.loads(data)
    
    filtered_data = []
    for item in json_data:
        filtered_item = FinancialKeyMetrics(
            symbol=item['symbol'],
            date=item['date'],
            grossProfitMargin=item.get('grossProfitMargin'),
            debtEquityRatio=item.get('debtEquityRatio'),
            debtRatio=item.get('debtRatio'),
            operatingProfitMargin=item.get('operatingProfitMargin'),
            netProfitMargin=item.get('netProfitMargin'),
            returnOnAssets=item.get('returnOnAssets'),
            returnOnEquity=item.get('returnOnEquity'),
            returnOnCapitalEmployed=item.get('returnOnCapitalEmployed'),
            totalDebtToCapitalization=item.get('totalDebtToCapitalization'),
            cashFlowCoverageRatios=item.get('cashFlowCoverageRatios'),
            quickRatio=item.get('quickRatio'),
            cashRatio=item.get('cashRatio'),
            assetTurnover=item.get('assetTurnover'),
            inventoryTurnover=item.get('inventoryTurnover'),
            receivablesTurnover=item.get('receivablesTurnover'),
            cashConversionCycle=item.get('cashConversionCycle'),
            priceEarningsRatio=item.get('priceEarningsRatio'),
            daysOfPayablesOutstanding=item.get('daysOfPayablesOutstanding')
        )
        filtered_data.append(filtered_item)
    
    return filtered_data[:observations]

# Function 3
# Will add more functions as needed



In [6]:
#State
from langgraph.graph.message import AnyMessage, add_messages

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    ticker: str
    observations: int
    financial_key_metrics: Optional[List[FinancialKeyMetrics]]
    financial_ratios: Optional[List[FinancialRatios]]
    company_summary: Optional[str]

In [16]:
from langchain_core.language_models import BaseLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser
from enum import Enum
from langchain_openai import ChatOpenAI

# Agent Decider
class QueryType(str, Enum):
    COMPANY_SPECIFIC = "CompanySpecific"
    GENERAL = "General"
    
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def agent_decider(state: State) -> dict:
    # Extract the last message
    last_message = state["messages"][-1] if state["messages"] else ""
    output_parser = EnumOutputParser(enum=QueryType)

    # Define the prompt
    prompt = ChatPromptTemplate.from_template(
        """You are an AI assistant that determines if a query is about a specific company or general knowledge.
        
        Query: {query}
        
        Is this query about a specific company or is it a general knowledge question?
        If it's about a specific company, respond with 'CompanySpecific'.
        If it's a general knowledge question, respond with 'General'.
        
        Your response should be only one of these two options."""
    )
    
    # Create a chain that uses the LLM and parses the output
    chain = prompt | llm | output_parser
    
    # Get the decision
    decision = chain.invoke({"query": last_message})
    
    return {"decision": decision}

In [17]:
# Test the agent_decider function

# Sample queries
test_queries = [
    "What are Apple's financial ratios?",
    "How do I calculate return on investment?",
    "Can you explain Tesla's recent stock performance?",
    "What is the difference between stocks and bonds?",
    "What was Amazon's revenue last quarter?"
]

# Create a mock state for testing
def create_mock_state(query):
    return State(
        messages=[{"role": "human", "content": query}],
        ticker="",
        observations=0,
        financial_key_metrics=None,
        financial_ratios=None,
        company_summary=None
    )

# Run tests
for query in test_queries:
    mock_state = create_mock_state(query)
    result = agent_decider(mock_state)
    print(f"Query: {query}")
    print(f"Classification: {result['decision']}")
    print("---")

Query: What are Apple's financial ratios?
Classification: QueryType.COMPANY_SPECIFIC
---
Query: How do I calculate return on investment?
Classification: QueryType.GENERAL
---
Query: Can you explain Tesla's recent stock performance?
Classification: QueryType.COMPANY_SPECIFIC
---
Query: What is the difference between stocks and bonds?
Classification: QueryType.GENERAL
---
Query: What was Amazon's revenue last quarter?
Classification: QueryType.COMPANY_SPECIFIC
---


In [ ]:
# Knowledge Base Retriever


In [ ]:
# Buffet Agent

In [ ]:
# Metric Extractor

In [ ]:
# Summarizer 

In [ ]:
# Graph